In [0]:
storage_account_name = "week17storageaccount"
container_name = "landing"
access_key = dbutils.secrets.get("databricks-scope", "databricksnewkey")

dbutils.fs.mount(
source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
mount_point="/mnt/landing",
extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)

True

In [0]:
storage_account_name = "week17storageaccount"
container_name = "reporting"
access_key = dbutils.secrets.get("databricks-scope", "databricksnewkey")

dbutils.fs.mount(
source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
mount_point="/mnt/reporting",
extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)

True

In [0]:
storage_account_name = "week17storageaccount"
container_name = "staging"
access_key = dbutils.secrets.get("databricks-scope", "databricksnewkey")

dbutils.fs.mount(
source=f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net",
mount_point="/mnt/staging",
extra_configs={f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": access_key}
)

True

In [0]:
dbutils.fs.ls("/mnt/staging/joined_df")

[FileInfo(path='dbfs:/mnt/staging/joined_df/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1714791214000),
 FileInfo(path='dbfs:/mnt/staging/joined_df/orders_year=2013/', name='orders_year=2013/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/staging/joined_df/orders_year=2014/', name='orders_year=2014/', size=0, modificationTime=0)]

In [0]:
df = spark.read.parquet("/mnt/staging/joined_df/")

In [0]:
df.show()

+--------+----------+-----------------+------------+--------------+--------------+----------------+-----------+--------------+-------------+------------+
|order_id|order_date|order_customer_id|orders_month|customer_fname|customer_lname|customer_zipcode|orders_year|customer_state|customer_city|order_status|
+--------+----------+-----------------+------------+--------------+--------------+----------------+-----------+--------------+-------------+------------+
|   25888|2014-01-01|             6735|           1|       Barbara|        Martin|           00725|       2014|            PR|       Caguas|    COMPLETE|
|   25895|2014-01-01|             1044|           1|         Linda|          Pope|           00725|       2014|            PR|       Caguas|    COMPLETE|
|   25897|2014-01-01|             6405|           1|          Mary|      Anderson|           00725|       2014|            PR|       Caguas|    COMPLETE|
|   25901|2014-01-01|             3099|           1|      Brittany|      Cop

In [0]:
from pyspark.sql.functions import *


In [0]:
df.select(expr("concat(customer_fname, ' ',customer_lname)").alias("customer_name"), "customer_city", "order_date", "order_status", "customer_state", "orders_year", "orders_month").createOrReplaceTempView("order_customer_details")

In [0]:
spark.sql("select * from order_customer_details limit 5").show()

+-----------------+-------------+----------+------------+--------------+-----------+------------+
|    customer_name|customer_city|order_date|order_status|customer_state|orders_year|orders_month|
+-----------------+-------------+----------+------------+--------------+-----------+------------+
|   Barbara Martin|       Caguas|2014-01-01|    COMPLETE|            PR|       2014|           1|
|       Linda Pope|       Caguas|2014-01-01|    COMPLETE|            PR|       2014|           1|
|    Mary Anderson|       Caguas|2014-01-01|    COMPLETE|            PR|       2014|           1|
|Brittany Copeland|       Caguas|2014-01-01|    COMPLETE|            PR|       2014|           1|
|        Mary Rowe|       Caguas|2014-01-01|    COMPLETE|            PR|       2014|           1|
+-----------------+-------------+----------+------------+--------------+-----------+------------+



In [0]:
spark.sql("cache table order_customer_details")

DataFrame[]

In [0]:
spark.sql("select distinct customer_name from order_customer_details where orders_year == 2014 and customer_state == 'TX' and order_status == 'COMPLETE'").show()

+-----------------+
|    customer_name|
+-----------------+
|   Elizabeth Ball|
|       Mary Patel|
|       Mary Baird|
|     Sandra Smith|
|  Christina Walsh|
|    Arthur Fisher|
|   Mary Patterson|
|     Henry Turner|
|    Patrick Smith|
|    Nancy Gilbert|
|     Austin Smith|
|Christopher Sloan|
| Raymond Richards|
|       Jose Smith|
|Christopher Smith|
|     Janice Novak|
|        Joe Smith|
|     Mary Bentley|
|     Dylan Butler|
|      Mary Ortega|
+-----------------+
only showing top 20 rows



In [0]:
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
dbutils.widgets.dropdown(name='orderstate',defaultValue='TX',choices=['TX','NY','PR'],label='STATE')


In [0]:
dbutils.widgets.dropdown(name='orderstatus',defaultValue='COMPLETE',choices=['COMPLETE','CLOSED','PROCESSING'],label='STATUS')

In [0]:
dbutils.widgets.text(name='orderyear',defaultValue="2014",label='YEAR')

In [0]:
state = dbutils.widgets.get("orderstate")
status = dbutils.widgets.get("orderstatus")
year = dbutils.widgets.get("orderyear")

In [0]:
spark.sql(f"select distinct customer_name from order_customer_details where customer_state == '{state}' AND order_status == '{status}' AND orders_year == '{year}'").show()

+-----------------+
|    customer_name|
+-----------------+
|   Elizabeth Ball|
|       Mary Patel|
|       Mary Baird|
|     Sandra Smith|
|  Christina Walsh|
|    Arthur Fisher|
|   Mary Patterson|
|     Henry Turner|
|    Patrick Smith|
|    Nancy Gilbert|
|     Austin Smith|
|Christopher Sloan|
| Raymond Richards|
|       Jose Smith|
|Christopher Smith|
|     Janice Novak|
|        Joe Smith|
|     Mary Bentley|
|     Dylan Butler|
|      Mary Ortega|
+-----------------+
only showing top 20 rows



In [0]:
print(state)

TX


In [0]:
print(status)

COMPLETE
